# Alignment-Based Compression of DNA Sequences

The compression of reads can be improved by aligning them to a known reference sequence.

**Experiment** &mdash; Sample 10 reads from the reference sequence `GATTTGGGGTTCAAAGCAGTATCGATCAAATAGTAAATCCATTTGTTCAACTCACAGTTTA`.

In [ ]:
import random

REF = "GATTTGGGGTTCAAAGCAGTATCGATCAAATAGTAAATCCATTTGTTCAACTCACAGTTTA"

NUM_READS = 10
MIN_READ_LEN = 5
MAX_READ_LEN = 10

reads = []

for i in range(NUM_READS):
    start = 0
    end = 0
    range_acquired = False
    while not range_acquired:
        start = random.randrange(len(REF))
        end = random.randrange(len(REF))
        if start > end:
            continue
        range_len = end - start + 1
        if range_len < MIN_READ_LEN:
            continue
        if range_len > MAX_READ_LEN:
            continue
        range_acquired = True
    read = REF[start : (end + 1)]
    reads.append(read)
    print(f"Read {i}: {read}")

**Experiment** &mdash; Add some noise to the sampled reads.

In [ ]:
def add_noise(read, abundance=4):
    choices = ["A", "C", "G", "T"]
    num_noisy_bases = 0
    noisy_read = ""
    for base in read:
        if not random.randrange(abundance):
            noisy_read += random.choice([c for c in choices if c != base])
            num_noisy_bases += 1
        else:
            noisy_read += base
    return noisy_read, num_noisy_bases


noisy_reads = []
for read in reads:
    noisy_read, num_noisy_bases = add_noise(read=read, abundance=4)
    print(f"Added {num_noisy_bases} noisy base(s): {read:10s} -> {noisy_read:10s}")
    noisy_reads.append(noisy_read)

**Experiment** &mdash; Align the noisy reads locally to the reference sequence. Compute the residual of each noisy read w.r.t. the reference sequence.

In [ ]:
import matplotlib.pyplot as plt
import alignment as aln

%matplotlib inline

residuals = []
total_num_clipped_bases = 0

for r in noisy_reads:
    align = aln.make_align(mode="local")
    r_aln, ref_aln, warp_path_r, warp_path_ref, scoring_mat = align(seq_a=r, seq_b=REF)

    print(f"Noisy read                 : {r}")
    aligned_noisy_read_without_gaps = list(filter(lambda elem: elem != "-", r_aln))
    print(f"Aligned noisy read w/o gaps: {''.join(aligned_noisy_read_without_gaps)}")
    num_clipped_bases = len(r) - len(aligned_noisy_read_without_gaps)
    print(f"Number of clipped bases    : {num_clipped_bases}")
    total_num_clipped_bases += num_clipped_bases
    ref_subsequence = REF[warp_path_ref[0] : (warp_path_ref[-1] + 1)]
    print(f"Reference subsequence      : {ref_subsequence}")

    residual = []
    for i, base in enumerate(r_aln):
        if i >= len(ref_subsequence) or base != ref_subsequence[i]:
            residual.append(base)
        else:
            residual.append("m")
    residuals.append(residual)
    print(f"Aligned noisy read         : {''.join(r_aln)}")
    print(f"Residual                   : {''.join(residual)}")

    plt.figure(figsize=[20, 10])
    plt.imshow(scoring_mat)
    plt.xticks(ticks=range(len(REF)), labels=REF)
    plt.yticks(ticks=range(len(r)), labels=r)
    plt.plot(warp_path_ref, warp_path_r, "w")
    plt.show()

print(f"In total, {total_num_clipped_bases} bases were clipped.")

**Experiment** &mdash; Calculate the entropy of the noisy reads and of the residual (plus an estimate for the clipped bases).

In [ ]:
import math
import entropy

noisy_reads = "".join(noisy_reads)
residuals = [elem for sublist in residuals for elem in sublist]
residuals = "".join(residuals)

noisy_reads_entropy = entropy.entropy(noisy_reads)
print(f"Noisy reads entropy: {round(noisy_reads_entropy, ndigits=2):.2f} bit/symbol")
residuals_entropy = entropy.entropy(residuals)
print(f"Residuals entropy: {round(residuals_entropy, ndigits=2):.2f} bit/symbol")
clipped_bases_entropy = (total_num_clipped_bases * 2) / total_num_clipped_bases
print(f"Clipped bases entropy: {clipped_bases_entropy:.2f} bit/symbol")

noisy_reads_size = math.ceil(noisy_reads_entropy * len(noisy_reads))
residuals_size = math.ceil(residuals_entropy * len(residuals))
clipped_bases_size = math.ceil(clipped_bases_entropy * total_num_clipped_bases)
print(f"Noisy reads size: {noisy_reads_size}")
print(f"Residuals size: {residuals_size}")
print(f"Clipped bases size: {clipped_bases_size}")